# Importing Modules

In [8]:
import pandas as pd
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input, Dense, ReLU
from keras import optimizers, layers
from tensorflow import math
import warnings
warnings.filterwarnings('ignore')

# Importing Data
- Data is of Rainfall in months of June to September of years 1948 - 2020

In [9]:
import statistics

rain = pd.read_csv("./Data/Rainfall.csv")

lpa = statistics.mean(list(rain['Jun-Sep'][12:41]))
rain_fall_data = list(rain['Jun-Sep'])

rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)

- Importing Sea-Level Pressure(SLP), Sea Surface Temperature(SST), and Zonal Wind(UWND) Data.

In [10]:
import glob

slp_path  = "./Data/Anomaly/slp/*"
sst_path  = "./Data/Anomaly/sst/*"
uwnd_path = "./Data/Anomaly/uwnd/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)

sst_months = []
for i in glob.glob(sst_path):
  sst_months.append(i)

uwnd_months = []
for i in glob.glob(uwnd_path):
  uwnd_months.append(i)
  
slp_months.sort()
sst_months.sort()
uwnd_months.sort()

- Start and End Index for data (i.e 1948's index and 2000's index)

In [11]:
s = 0
e = 52

- Extracting data of all the months and stacking one above other. 
- 324 variables and 52 data points for SLP and UWND.
- 192 variables and 52 data points for SST.

In [15]:
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d))
  return np.array(new_data)

# Stacking Data of All Months
combine_slp, combine_sst, combine_uwnd = [], [], []
for i in range(12):
  month_path_slp, month_path_sst, month_path_uwnd = slp_months[i], sst_months[i], uwnd_months[i] 
  month_data_slp ,month_data_sst, month_data_uwnd = pd.read_csv(month_path_slp, header=[0, 1], index_col=0), pd.read_csv(month_path_sst, header=[0, 1], index_col=0), pd.read_csv(month_path_uwnd, header=[0, 1], index_col=0)
 
  feature_slp, feature_sst, feature_uwnd = get_feature(month_data_slp), get_feature(month_data_sst), get_feature(month_data_uwnd)
  feature_slp, feature_sst, feature_uwnd = feature_slp[s:e], feature_sst[s:e], feature_uwnd[s:e]
  for i in feature_slp:
    combine_slp.append(i)
  for i in feature_sst:
    combine_sst.append(i)
  for i in feature_uwnd:
    combine_uwnd.append(i)

combine = [np.array(combine_slp), np.array(combine_sst), np.array(combine_uwnd)]
month_data_slp.head()

Latitude         -90                                                         \
Longitude          0         20         40         60         80        100   
1949       -3.649323  -3.649323  -3.649323  -3.649323  -3.649323  -3.649323   
1950        1.230677   1.230677   1.230677   1.230677   1.230677   1.230677   
1951        7.090677   7.090677   7.090677   7.090677   7.090677   7.090677   
1952        8.300677   8.300677   8.300677   8.300677   8.300677   8.300677   
1953       10.240677  10.240677  10.240677  10.240677  10.240677  10.240677   

Latitude                                               ...        80  \
Longitude        120        140        160        180  ...       160   
1949       -3.649323  -3.649323  -3.649323  -3.649323  ...  6.598676   
1950        1.230677   1.230677   1.230677   1.230677  ... -4.251374   
1951        7.090677   7.090677   7.090677   7.090677  ...  0.478676   
1952        8.300677   8.300677   8.300677   8.300677  ...  8.528676   
1953       10.240677  10.240677  10.240677  10.240677  ... -1.301354   

Latitude                                                               \
Longitude       180       200       220       240       260       280   
1949       5.294682  3.563006  1.995687  1.005682  0.951014  1.386347   
1950      -4.835348 -5.186994 -4.854313 -2.824318 -0.829026 -1.153683   
1951       3.114682  5.042976  5.235657  5.285642  5.451014  5.246347   
1952       9.234682  9.053006  7.765687  5.885682  4.100964  2.876347   
1953      -1.155358 -1.876994 -3.864313 -5.174318 -4.628986 -2.843693   

Latitude                                
Longitude      300       320       340  
1949       1.43035  1.857351  3.877972  
1950      -1.92965 -1.202699  0.617962  
1951       5.95035  6.487301  4.937972  
1952       2.64035  3.047301  2.927962  
1953      -2.32968 -3.162649 -2.762038  

[5 rows x 324 columns]

- Concanating the Data as required.

In [17]:
def get_data(List):
    N = len(List)
    if N == 1:
        return combine[List[0]]
    elif N == 2:
        return np.concatenate((combine[List[0]], combine[List[1]]), axis=1)
    else:
        return np.concatenate((combine[List[0]], combine[List[1]], combine[List[2]]), axis=1)


# Models 

## Auto-Encoders (SLP or UWND)

- 324 - 97 - 323

In [18]:
def get_model_324(n_inputs=324):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(97)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 97 - 29 - 97

In [19]:
def get_model_97(n_inputs=97):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(29)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 29 - 9 - 29

In [20]:
def get_model_29(n_inputs=29):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(9)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

#### Final Auto Encoder
- 324 - 97 - 29 - 9 - 29 - 97 - 324

In [21]:
def get_model_final_324(n_inputs=324):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    e = Dense(97)(e)
    e = ReLU()(e)
    e = Dense(29)(e)
    e = ReLU()(e)
    bottleneck = Dense(9)(e)
    e = Dense(29)(bottleneck)
    e = ReLU()(e)
    e = Dense(97)(e)
    e = ReLU()(e)
    output = Dense(n_inputs, activation='linear')(e)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

## Auto-Encoders (SST)

- 192 - 72 - 192

In [22]:
def get_model_192(n_inputs=192):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(72)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 72 - 21 - 72

In [23]:
def get_model_72(n_inputs=72):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(21)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 21 - 6 - 21

In [24]:
def get_model_21(n_inputs=21):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(6)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

#### Final Auto Encoder
- 192 - 72 - 21 - 6 - 21 - 72 - 192

In [25]:
def get_model_final_192(n_inputs=192):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    e = Dense(72)(e)
    e = ReLU()(e)
    e = Dense(21)(e)
    e = ReLU()(e)
    bottleneck = Dense(6)(e)
    e = Dense(21)(bottleneck)
    e = ReLU()(e)
    e = Dense(72)(e)
    e = ReLU()(e)
    output = Dense(n_inputs, activation='linear')(e)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

## Auto-Encoders (SST-SLP or SST-UWND)

- 416 - 169 - 416

In [26]:
def get_model_416(n_inputs=416):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(169)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 169 - 50 - 169

In [27]:
def get_model_169(n_inputs=169):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(50)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 50 - 15 - 50

In [28]:
def get_model_50(n_inputs=50):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(15)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

#### Final Auto Encoder
- 416 - 169 - 50 - 15 - 50 - 169 - 416

In [29]:
def get_model_final_416(n_inputs=416):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    e = Dense(169)(e)
    e = ReLU()(e)
    e = Dense(21)(e)
    e = ReLU()(e)
    bottleneck = Dense(6)(e)
    e = Dense(21)(bottleneck)
    e = ReLU()(e)
    e = Dense(169)(e)
    e = ReLU()(e)
    output = Dense(n_inputs, activation='linear')(e)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

## Auto-Encoders (SLP-UWND)

- 648 - 194 - 648

In [30]:
def get_model_648(n_inputs=648):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(194)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 194 - 58 - 194

In [31]:
def get_model_194(n_inputs=194):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(58)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 58 - 17 - 58

In [32]:
def get_model_58(n_inputs=58):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(17)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

#### Final Auto Encoder
- 648 - 194 - 58 - 17 - 58 - 194 - 648

In [33]:
def get_model_final_648(n_inputs=648):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    e = Dense(194)(e)
    e = ReLU()(e)
    e = Dense(58)(e)
    e = ReLU()(e)
    bottleneck = Dense(17)(e)
    e = Dense(58)(bottleneck)
    e = ReLU()(e)
    e = Dense(194)(e)
    e = ReLU()(e)
    output = Dense(n_inputs, activation='linear')(e)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

## Get all Models in a 2-D List
|       Outer Most Layer           |        Middle Layer             |      Inner Most Layer         |      Full Model                              |
|----------------------------------|---------------------------------|-------------------------------|----------------------------------------------|
| [ Model SLP  - 324-97-324        |  Model SLP  - 97-29-97          |  Model SLP  - 29-9-29         |  Model SLP  - 324-97-29-9-29-97-324  ]       |
| [ Model UWND - 324-97-324        |   Model UWND - 97-29-97         |   Model UWND - 29-9-29        |   Model UWND - 324-97-29-9-29-97-324 ]       |
| [ Model SST  - 192-72-192        |   Model SST  - 72-21-72         |   Model SST  - 21-6-21        |   Model SST  - 192-72-21-6-21-72-192 ]       |
| [ Model SLP-SST  - 416-169-416   |   Model SLP-SST  - 169-50-169   |   Model SLP-SST  - 50-15-50   |   Model SLP-SST  - 416-169-50-15-50-169-416] |
| [ Model UWND-SST - 416-169-416   |   Model UWND-SST - 169-50-169   |   Model UWND-SST - 50-15-50   |   Model UWND-SST - 416-169-50-15-50-169-416] |
| [ Model SLP-UWND - 648-194-648   |   Model SLP-UWND - 194-58-194   |   Model SLP-UWND - 58-17-58   |   Model SLP-UWND - 416-194-58-17-58-194-416] |

In [34]:
def get_all_model():
    models_slp  = [get_model_324(), get_model_97(), get_model_29(), get_model_final_324()]
    models_uwnd = [get_model_324(), get_model_97(), get_model_29(), get_model_final_324()]
    models_sst  = [get_model_192(), get_model_72(), get_model_21(), get_model_final_192()]
    models_slp_sst  = [get_model_416(), get_model_169(), get_model_50(), get_model_final_416()]
    models_uwnd_sst = [get_model_416(), get_model_169(), get_model_50(), get_model_final_416()]
    models_slp_uwnd = [get_model_648(), get_model_194(), get_model_58(), get_model_final_648()]

    models = [models_slp, models_uwnd, models_sst, models_slp_sst, models_uwnd_sst, models_slp_uwnd]
    return models

In [35]:
models = get_all_model()

In [36]:
for i in models[0]:
    i.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense (Dense)               (None, 324)               105300    
                                                                 
 re_lu (ReLU)                (None, 324)               0         
                                                                 
 dense_1 (Dense)             (None, 97)                31525     
                                                                 
 dense_2 (Dense)             (None, 324)               31752     
                                                                 
Total params: 168,577
Trainable params: 168,577
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
______________________________________

# Training

In [38]:
data = np.concatenate((get_data([0]), get_data([2])), axis=0)
data.shape

(1248, 324)

In [41]:
def train_model(models, inputs):
    x = inputs
    for model in models[:-1]:
        model.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))
        model_temp = Model(x=model.input, outputs=model.layers[3].output)
        x = model_temp(x)

    model[-1].layers[2] = model[0].layers[3]
    model[-1].layers[11] = model[0].layers[4]
    model[-1].layers[4] = model[1].layers[3]
    model[-1].layers[9] = model[1].layers[4]
    model[-1].layers[6] = model[2].layers[3]
    model[-1].layers[7] = model[2].layers[4]
    model[-1].compile(optimizer='adam', loss='mse')

    model[-1].fit(inputs, inputs, epochs=800, batch_size=16, verbose=2, validation_data=(inputs,inputs))

In [42]:
train_model(models[0], data)

Epoch 1/800


UnimplementedError: Graph execution error:

Detected at node 'model/Cast' defined at (most recent call last):
    File "C:\Users\Sahaj\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "C:\Users\Sahaj\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "C:\Users\Sahaj\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\Sahaj\AppData\Roaming\Python\Python37\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Sahaj\AppData\Roaming\Python\Python37\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Sahaj\AppData\Roaming\Python\Python37\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\